# Показатели каналов объединенных в группу
Пример расчета показателей по группе телеканалов.

## Описание задачи и условий расчета
- Период: 03.08.2022 - 09.08.2022
- Временной интервал: 05:00-29:00
- ЦА: Все 14-44
- Место просмотра: Все места (дом+дача)
- Каналы: ТНТ, Пятница, ТВ-3, 2х2, ТНТ 4, Суббота, Матч ТВ (в группе и отдельно)									
- Статистики: AvReach000

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Справочники

In [2]:
# выберем телесети по имени (части имени) и получим их id
cats.get_tv_net(name=["ТНТ", "ПЯТНИЦА", "ТВ-3", "2x2", "ТНТ 4", "СУББОТА", "МАТЧ ТВ"])

# далее в запросах будем использовать id телесетей.

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [3]:
# Найдем названия нужных нам демографических переменных
cats.get_tv_demo_attribute(names=['возраст'])

Запрошены записи: 0 - 125
Всего найдено записей: 125



,id,valueId,name,colName,valueName
0,2,33,Возраст,Age,33
1,16,1,Возраст детей: До 1 года,KidsAge2,Да
2,16,0,Возраст детей: До 1 года,KidsAge2,Нет
3,20,0,Возраст детей: 7-11 лет,KidsAge6,Нет
4,20,1,Возраст детей: 7-11 лет,KidsAge6,Да
...,...,...,...,...,...
120,2,10,Возраст,Age,10
121,2,89,Возраст,Age,89
122,2,8,Возраст,Age,8
123,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


___________________________________________________________________________________________

Разделим задачу на два задания: 
- по каждому каналу в отдельности
- по группе каналов

и затем объединим результат в одну таблицу

## По каждому каналу в отдельности

### Формирование задания

Зададим условия расчета

In [4]:
# Период указывается в виде списка ('Начало', 'Конец'). 
date_filter = [('2022-08-03', '2022-08-09')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА
basedemo_filter = 'age >= 14 AND age <= 44'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvNetId IN (326, 376, 286, 83, 204, 329, 206)'

# Указываем список статистик для расчета
statistics = ['AvReach000']

# Указываем срезы - разбиваем по каналу
slices = ['tvNetName']

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

### Расчет задания

In [5]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_task('timeband', date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df_chann = mtask.result2table(mtask.get_result(task_timeband))
df_chann

Расчет задачи (id: 662abc42-22b3-4664-9886-54556adc0eec) [= = ] время расчета: 0:00:07.653852


,tvNetName,AvReach000
0,МАТЧ ТВ,1979.364060
1,2X2,883.768030
2,ТНТ 4,1061.163073
3,ТВ-3,4852.995402
4,СУББОТА,1400.820543
5,ПЯТНИЦА,4675.625814
6,ТНТ,6451.462758


## По группе каналов

### Формирование задания
Зададим условия расчета

In [6]:
# Для расчета по группе каналов уберем срез по каналам и оставим это поле пустым
slices = []
# Другие условия расчета не меняются

### Расчет задания

In [7]:
# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task('timeband', date_filter=date_filter, 
                                 weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                 company_filter=company_filter, time_filter=time_filter, 
                                 basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                 location_filter=location_filter, slices=slices, 
                                 statistics=statistics, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df_group = mtask.result2table(mtask.get_result(task_timeband))
df_group.insert(0, 'tvNetName', 'ТНТ, Пятница, ТВ-3, 2х2, ТНТ 4, Суббота, Матч ТВ') #Добавляем колонку с названием группы
df_group

Расчет задачи (id: 12a8fbde-9e0b-40c7-87f6-6b51b1f587d9) [= ] время расчета: 0:00:04.138328


,tvNetName,AvReach000
0,"ТНТ, Пятница, ТВ-3, 2х2, ТНТ 4, Суббота, Матч ТВ",13740.259061


## Объединяем две таблицы в одну

In [8]:
df = pd.concat([df_chann, df_group], ignore_index=True)
df

,tvNetName,AvReach000
0,МАТЧ ТВ,1979.364060
1,2X2,883.768030
2,ТНТ 4,1061.163073
3,ТВ-3,4852.995402
4,СУББОТА,1400.820543
5,ПЯТНИЦА,4675.625814
6,ТНТ,6451.462758
7,"ТНТ, Пятница, ТВ-3, 2х2, ТНТ 4, Суббота, Матч ТВ",13740.259061


## Экспорт в Excel
По умолчанию файл сохраняется в директорию `mediascope-jupyter\mediascope-notebooks\excel`

In [9]:
writer = pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_joined_channels'))
df_info = mtask.task_builder.get_report_info()
df.to_excel(writer, 'Report', index=True)
df_info.to_excel(writer, 'Info', index=False)
writer.save()